In [21]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms, models
import torch.nn.functional as F
from sklearn.datasets import fetch_olivetti_faces
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
data = fetch_olivetti_faces(shuffle=True, random_state=0)
images = data.images.reshape((-1, 1, 64, 64)).astype(np.float32)  # Reshape to [N, C, H, W]
targets = data.target

# Split the dataset into training and testing
train_images, test_images = images[:300], images[300:]
train_targets, test_targets = targets[:300], targets[300:]

# Convert numpy arrays to PyTorch tensors
train_images_tensor = torch.tensor(train_images)
train_targets_tensor = torch.tensor(train_targets)
test_images_tensor = torch.tensor(test_images)
test_targets_tensor = torch.tensor(test_targets)

# Create DataLoader
train_dataset = TensorDataset(train_images_tensor, train_targets_tensor)
test_dataset = TensorDataset(test_images_tensor, test_targets_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [3]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, activation_fn=nn.ReLU, drop_rate=0.0):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.activation_fn = activation_fn(inplace=True)
        self.dropout = nn.Dropout(p=drop_rate)  # Dropout layer
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = self.activation_fn(self.bn1(self.conv1(x)))
        out = self.dropout(out)  # Apply dropout
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.activation_fn(out)
        return out

    
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=40, activation_fn=nn.ReLU):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # Initial convolution
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.activation_fn = activation_fn(inplace=True)
        
        # ResNet layers
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1, activation_fn=activation_fn,drop_rate=0.1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2, activation_fn=activation_fn,drop_rate=0.1)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2, activation_fn=activation_fn,drop_rate=0.1)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2, activation_fn=activation_fn,drop_rate=0.1)

        self._to_linear = None
        self._get_conv_output_size(torch.rand(1, 1, 64, 64))

        self.linear = nn.Linear(self._to_linear, num_classes)
    
    def _make_layer(self, block, planes, num_blocks, stride, activation_fn, drop_rate=0.0):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, activation_fn=activation_fn, drop_rate=drop_rate))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)


    def _get_conv_output_size(self, shape):
        with torch.no_grad():
            self.eval()
            output = self._forward_conv_only(shape)
            self._to_linear = np.prod(output.size()[1:])
            self.train()

    def _forward_conv_only(self, x):
        out = self.activation_fn(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        return out

    def forward(self, x):
        out = self._forward_conv_only(x)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


In [4]:
# Instantiate the model
num_blocks = [2, 2, 2, 2]  # ResNet18-like architecture
model = ResNet(BasicBlock, num_blocks, num_classes=40)

In [17]:
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
        
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        
        train_accuracy = 100 * correct_train / total_train
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, '
              f'Train Accuracy: {train_accuracy:.2f}%')
        
        # Evaluation phase
        model.eval()  # Set model to evaluation mode
        correct_test = 0
        total_test = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()
        
        test_accuracy = 100 * correct_test / total_test
        print(f'Test Accuracy: {test_accuracy:.2f}%\n')


# Now you can call the training function with the necessary parameters
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=40)


Epoch 1/40, Loss: 0.0098, Train Accuracy: 99.67%
Test Accuracy: 80.00%

Epoch 2/40, Loss: 0.0381, Train Accuracy: 99.33%
Test Accuracy: 57.00%

Epoch 3/40, Loss: 0.0413, Train Accuracy: 99.00%
Test Accuracy: 62.00%

Epoch 4/40, Loss: 0.0121, Train Accuracy: 100.00%
Test Accuracy: 78.00%

Epoch 5/40, Loss: 0.0185, Train Accuracy: 99.67%
Test Accuracy: 84.00%

Epoch 6/40, Loss: 0.0040, Train Accuracy: 100.00%
Test Accuracy: 84.00%

Epoch 7/40, Loss: 0.0105, Train Accuracy: 99.67%
Test Accuracy: 87.00%

Epoch 8/40, Loss: 0.0670, Train Accuracy: 98.67%
Test Accuracy: 80.00%

Epoch 9/40, Loss: 0.0267, Train Accuracy: 98.67%
Test Accuracy: 65.00%

Epoch 10/40, Loss: 0.0398, Train Accuracy: 98.67%
Test Accuracy: 33.00%

Epoch 11/40, Loss: 0.0180, Train Accuracy: 99.33%
Test Accuracy: 40.00%

Epoch 12/40, Loss: 0.0216, Train Accuracy: 99.33%
Test Accuracy: 52.00%

Epoch 13/40, Loss: 0.0119, Train Accuracy: 99.33%
Test Accuracy: 69.00%

Epoch 14/40, Loss: 0.0345, Train Accuracy: 99.00%
Test Acc

In [30]:
model.eval()

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation_fn): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation_fn): ReLU(inplace=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation_fn): ReLU

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
correct = 0
total = 0
with torch.no_grad():  # Disable gradient tracking
    for images, labels in test_loader:  # Assuming test_loader is your test dataset DataLoader
        images, labels = images.to(device), labels.to(device)  # Move to the correct device
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')


Test Accuracy: 94.00%
